In [2]:
%pip install faiss-cpu -U sentence-transformers mistralai python-docx

Defaulting to user installation because normal site-packages is not writeable
     |█▎                              | 10 kB 14.7 MB/s eta 0:00:01  Downloading python_docx-1.2.0-py3-none-any.whl (252 kB)
     |████████████████████████████████| 252 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 252 kB 5.2 MB/s eta 0:00:01
     |██▍                             | 634 kB 52.0 MB/s eta 0:00:01Collecting lxml>=3.1.0
     |████████████████████████████████| 8.6 MB 52.0 MB/s eta 0:00:01
     |████████████████████████████████| 8.6 MB 52.0 MB/s 
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.




## Read csv


In [3]:
from docx import Document

def extract_text_from_docx(file_path):
    """
    Extract all text from a Word document (.docx file).
    
    Args:
        file_path: Path to the .docx file
        
    Returns:
        String containing all text from the document
    """
    all_text = []
    try:
        doc = Document(file_path)
        
        # Extract text from all paragraphs
        for para in doc.paragraphs:
            if para.text.strip():  # Only add non-empty paragraphs
                all_text.append(para.text)
        
        # Extract text from tables if any
        for table in doc.tables:
            for row in table.rows:
                row_text = []
                for cell in row.cells:
                    row_text.append(cell.text.strip())
                all_text.append(" | ".join(row_text))
        
        return "\n".join(all_text)
    except FileNotFoundError:
        return "Error: File not found."
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage:
all_text = extract_text_from_docx('/Users/harveys/Desktop/repos/BOchat/Box Office Manual (2025) - working.docx')

In [4]:
print(all_text[:100])
print(len(all_text))






BOX OFFICE MANUAL
 ~2026~



























VENUE INFORMATION 

Melbourne Town Hall (M
109612


## Semantic Chunking

Instead of fixed-size character chunks, we use semantic chunking that respects sentence boundaries. This preserves meaning and context while maintaining manageable chunk sizes.

In [5]:
import re
from typing import List

def semantic_chunking(text: str, max_chunk_size: int = 800, overlap: int = 100) -> List[str]:
    """
    Split text into semantically meaningful chunks based on sentence boundaries.
    
    Args:
        text: The full text to chunk
        max_chunk_size: Maximum characters per chunk
        overlap: Number of characters to overlap between chunks
        
    Returns:
        List of text chunks
    """
    # Split by sentences (handles periods, exclamation marks, question marks)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # Add sentence to current chunk if it fits
        if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:
            current_chunk += " " + sentence if current_chunk else sentence
        else:
            # Save current chunk and start a new one
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = sentence
    
    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk)
    
    # Add overlap between chunks for context continuity
    overlapped_chunks = []
    for i, chunk in enumerate(chunks):
        if i > 0:
            # Add end of previous chunk for context
            prev_chunk_overlap = chunks[i-1][-overlap:] if len(chunks[i-1]) > overlap else chunks[i-1]
            overlapped_chunks.append(prev_chunk_overlap + " " + chunk)
        else:
            overlapped_chunks.append(chunk)
    
    return overlapped_chunks

chunks = semantic_chunking(all_text, max_chunk_size=800, overlap=100)
print(f"Number of chunks: {len(chunks)}")
print(f"Average chunk size: {sum(len(c) for c in chunks) / len(chunks):.0f} characters")

Number of chunks: 157
Average chunk size: 795 characters


we split documents into smaller chunks so it's more effective to identify and retrieve the most relevant information.


## embed


for each text chunk, we create a text embedding, which are numerical representations of text in the vector space, where words with similar meanings are closer to each other in this space.


In [6]:
import numpy as np

model_id = "sentence-transformers/all-MiniLM-L6-v2"
#hf_token = "{Token}"


#run embedding model locally
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_id)

def embed(text):
    output = model.encode(text)
    return output

embeddings = embed(chunks)

embeddings
dimension = embeddings.shape[1]

/Users/harveys/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/harveys/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/harveys/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
#call embedding model from Huggingface API
from huggingface_hub import InferenceClient
import os

def embed(text):
    client = InferenceClient(model="sentence-transformers/all-MiniLM-L6-v2", token=hf_token)
    output = client.feature_extraction(text)
    return output

embeddings = embed(chunks)


In [7]:
embeddings

array([[ 0.0423224 , -0.04869147,  0.00018905, ...,  0.05051366,
        -0.0878363 ,  0.04072192],
       [ 0.09235675, -0.04075618, -0.02773448, ...,  0.04207275,
        -0.1127654 ,  0.00519531],
       [ 0.09721361, -0.03043868, -0.04293957, ..., -0.02132797,
        -0.1775212 ,  0.02543559],
       ...,
       [-0.04611181, -0.00928476, -0.03302702, ..., -0.01528093,
        -0.0166852 ,  0.01827378],
       [-0.01382715, -0.00236359, -0.00318398, ..., -0.0137355 ,
        -0.0401839 , -0.00121509],
       [-0.05528684, -0.0287168 , -0.0746312 , ...,  0.03533925,
         0.01555696, -0.04573799]], dtype=float32)

In [24]:
dimension

384

In [9]:
import pandas as pd
embeddings_df = pd.DataFrame(embeddings)
embeddings_df.to_csv("embeddings.csv", index=False)

## Storeing embedding vectors for retrieval


once we have the embeddings, we store them in a vector db for efficient processing and retrieval - using faiss here

In [25]:
import faiss

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)

## query


when the user asks a question, it is also embedded using the function from above.


In [26]:
question = "Where is Melbourne Town Hall?"
question_embeddings = np.array([embed(question)])

## retrieval


we perform search on our vector db using `index.search`, it takes two parameters, the embedding of our question and k, which is the number of similar vectors to retrieve

The function returns the distances (D) and indices (I) of the most similar vector, and based on the indices, we can return the actual text.


In [27]:
D, I = index.search(question_embeddings, k=2)  # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

## Create prompt


we create a prompt template that combines the retrieved chunk that contains the most relevant/simalar info and then the users question


In [28]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

## Chat model


We then pass this prompt to the LLM and generate an answer based on the user's question and the context retrieved from our documentation. Here we're using Mistral's 'mistral-medium' model via the Mistral API


In [29]:
from mistralai.client import MistralClient
from mistralai import Mistral

client = Mistral(api_key="API_KEY")

def run_mistral(user_message, model="mistral-medium"):
    messages = [{"role":"user", "content":user_message}]
    chat_response = client.chat.complete(model=model, messages=messages)
    return chat_response.choices[0].message.content


run_mistral(prompt)

'Melbourne Town Hall is located at 90/130 Swanston St, Melbourne VIC 3000.'

## all together now


In [17]:
from faiss import IndexFlatL2

prompt = """
Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query}
Answer:
"""


def ask(query: str, index: IndexFlatL2, chunks):
    embedding = embed(query)
    embedding = np.array([embedding])

    _, indexes = index.search(embedding, k=2)
    context = [chunks[i] for i in indexes.tolist()[0]]

    user_message = prompt.format(context=context, query=query)

    messages = [{"role":"user", "content":user_message}]
    chat_response = client.chat.complete(model="mistral-medium", messages=messages)
    return chat_response.choices[0].message.content


ask("What do i need to bring to Plenary for a venue shift?", index, chunks)

'The context information provided does not mention anything about "Plenary" or what needs to be brought for a venue shift there. The details cover venues like Melbourne Town Hall (MTH), Hamer Hall, and Sidney Myer Music Bowl (SMMB), but there is no reference to a venue named "Plenary."\n\nIf you are referring to a different venue or if "Plenary" is a typo or alternative name for one of the mentioned venues, please clarify or provide additional context. Otherwise, I cannot provide an answer based on the given information.'

In [44]:
ask("How do i add an e-membership?", index, chunks)

'To add an e-membership to a member\'s constituency, follow these steps:\n\n1. **Access the Member\'s Profile**: Log in to the Tessi system and locate the member\'s profile.\n2. **Navigate to the Attributes Tab**: Once in the member\'s profile, go to the "Attributes" tab.\n3. **Select the e-Membership Attribute**: In the Attributes tab, find and select the attribute labeled "eMembership."\n4. **Set the Attribute Value**: Set the attribute value to "Yes."\n5. **Save and Close**: Save the changes and close the profile.\n\nThis process ensures that the member\'s welcome pack and other membership materials are sent digitally. You can verify if a digital membership pack has been sent by checking the "e-membership" attribute in the member\'s Tessi account under the Attributes tab.'

In [45]:
ask("Are there steps in Hamer Hall Stalls?", index, chunks)

'The context information does not explicitly mention the presence of steps in the Stalls section of Hamer Hall. However, it does provide details about the location of Swing Arm Seats in the Stalls section, which are located on the aisle seats in specific rows (B, E, H, L, P, T, and W). This suggests that there are aisles and possibly steps between rows, but it is not explicitly confirmed.\n\nFor more detailed information, you might need to refer to the seat map or additional resources provided in the context, such as the link to the Hamer Hall seat map.'

In [46]:
ask("How do installment plans work?", index, chunks)

'Based on the provided context, here’s how installment plans work:\n\n1. **Payment Schedule**: Installment plans allow customers to split payments over multiple dates. The **Instalment Payment Date Calculator** (linked in the context) is used to determine the payment dates, including any applicable grace periods.\n\n2. **Late Payments**:\n   - If a payment fails, a **$10 late fee** is added.\n   - Customers are contacted to correct the payment error by the following Thursday to avoid subscription cancellation.\n   - If no response is received by the following Friday, the subscription may be canceled, with a refund issued minus fees (including the $10 late fee).\n\n3. **Flexibility for Missed Payments**:\n   - Changes to installment payment dates can be made if individual circumstances prevent payment for a specific month.\n   - Only the failed payment date can be adjusted; subsequent scheduled payments remain unchanged.\n   - If a payment date is changed, **Nicole** must be notified to